In [ ]:
#importando bibliotecas
import os
import base64
import requests
import pandas as pd

#grafico
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lendo os arquivos
df_docente = pd.read_csv("DM_DOCENTE.CSV")
df_aluno = pd.read_csv("DM_ALUNO.CSV")
#df_variaveis = pd.read_excel("Dicionário de Variáveis.xls", "DM_ALUNO")
df_codigo = pd.read_excel("Código_do_País_de_Nascimento_ou_Naturalização.xlsx")
df_disciplinas = pd.read_csv("dados_disciplinas.txt", sep=",")

In [ ]:
df_disciplinas[(df_disciplinas["SITUACAO"] == "Aprovado")]

NameError: name 'df_disciplinas' is not defined

In [2]:
# plotar correlação 

corr = df_disciplinas[["MATR_ALUNO","ANO","PERIODO","SITUACAO","MEDIA_FINAL","NUM_FALTAS","DISCIPLINA"]][(df_disciplinas["SITUACAO"]=="APROVADO")].corr(numeric_only=True)
print(corr) 

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlação")
plt.show()

NameError: name 'df_disciplinas' is not defined